**Import dependecies**

In [1]:

%conda install -c apple tensorflow-deps scipy #macos
%pip install tensorflow-macos tensorflow-metal #macos
'''
%pip install --upgrade tensorflow 
%pip install pydot graphviz
'''
%pip install pandas numpy pillow matplotlib scipy autopep8 pydot


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext tensorboard

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
import os
from tensorflow import keras
from keras.applications.vgg16 import VGG16
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model
import datetime

fruit = "apple"
model_current = "vgg16"
curr_datetime = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
run_prefix = f'./runs/{fruit}/{model_current}/{curr_datetime}/'
results_prefix = f'./results/{fruit}/{model_current}/{curr_datetime}/'
log_dir = f'logs/fit/{fruit}/{model_current}/{curr_datetime}'
os.makedirs(run_prefix, exist_ok=True)
os.makedirs(results_prefix, exist_ok=True)
run_prefix


'./runs/apple/vgg16/20220727-175321/'

**Create an imagedatagenerator object to label data**

In [6]:
input_size = 128
batch_size = 16
train_size = 8000
test_size = 2000
learning_rate = 0.001
epochs = 200
dropout = 0.5
dense_size = 4096

train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
train_dir = f"./dataset_frutas_colab/{fruit}/train"
test_dir = f"./dataset_frutas_colab/{fruit}/test"
train_generator = train_datagen.flow_from_directory(train_dir,batch_size=batch_size,class_mode = 'binary', target_size=(input_size,input_size), shuffle= True)
test_generator = test_datagen.flow_from_directory(test_dir,batch_size=batch_size,class_mode = 'binary', target_size=(input_size,input_size), shuffle= True)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


VGG16 model

In [7]:
pre_trained_model = VGG16(
    input_shape = (input_size,input_size,3),
    include_top = False
)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-07-27 17:53:21.537805: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-27 17:53:21.538156: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Start transfer learning process

In [8]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [9]:
import tensorflow as tf
#Flatten
x = layers.Flatten()(pre_trained_model.output)
#Fully connected layer con 1,024 hidden units y ReLU
x = layers.Dense(dense_size, activation='relu')(x)
#Dropout rate
x = layers.Dropout(dropout)(x)
#Función de clasificación sigmoidal - Clasificación binaria
x = layers.Dense(1,activation='sigmoid')(x)

model = Model(pre_trained_model.input,x)

model.compile(optimizer = Adam(learning_rate=learning_rate),
              loss='binary_crossentropy',
              metrics = [tf.keras.metrics.Accuracy(),
                         tf.keras.metrics.Precision(), 
                         tf.keras.metrics.Recall()])

In [10]:
tf.keras.utils.plot_model(
    model,
    to_file=f"{run_prefix}model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=True,
    dpi=96,
    layer_range=None,
    show_layer_activations=True,
)
print("Saved Model Image")

Saved Model Image


In [11]:
import datetime

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(
    train_generator,
    validation_data = test_generator,
    steps_per_epoch = train_size/batch_size,
    epochs = epochs,
    validation_steps = test_size/batch_size,
    callbacks=[tensorboard_callback]
)

Epoch 1/200


2022-07-27 17:53:22.685797: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-27 17:53:23.198873: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


500/500 [==============================] - ETA: 0s - loss: 0.2837 - accuracy: 0.0010 - precision: 0.8997 - recall: 0.9108

2022-07-27 17:53:53.769583: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


500/500 [==============================] - 49s 97ms/step - loss: 0.2837 - accuracy: 0.0010 - precision: 0.8997 - recall: 0.9108 - val_loss: 0.1915 - val_accuracy: 0.0000e+00 - val_precision: 0.9976 - val_recall: 0.8380
Epoch 2/200
500/500 [==============================] - 47s 94ms/step - loss: 0.1127 - accuracy: 0.0019 - precision: 0.9552 - recall: 0.9600 - val_loss: 0.3703 - val_accuracy: 0.0000e+00 - val_precision: 1.0000 - val_recall: 0.6610
Epoch 3/200
500/500 [==============================] - 46s 91ms/step - loss: 0.0889 - accuracy: 0.0055 - precision: 0.9661 - recall: 0.9680 - val_loss: 0.2832 - val_accuracy: 0.0000e+00 - val_precision: 0.9987 - val_recall: 0.7420
Epoch 4/200
500/500 [==============================] - 46s 92ms/step - loss: 0.0654 - accuracy: 0.0165 - precision: 0.9758 - recall: 0.9785 - val_loss: 0.2676 - val_accuracy: 0.0000e+00 - val_precision: 1.0000 - val_recall: 0.7810
Epoch 5/200
500/500 [==============================] - 46s 92ms/step - loss: 0.0550 - ac

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
precision = history.history['precision']
recall = history.history['recall']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.savefig(f"{run_prefix}acc.svg")
plt.figure()


plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig(f"{run_prefix}loss.svg")

plt.show()

In [ ]:
#model.save(f"{run_prefix}{model_current}.h5")

In [ ]:
import csv

header = ["train_acc", "val_acc", "train_loss", "val_loss", "precision", "recall"]

with open(f'{results_prefix}metrics.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)
    for idx in range(len(acc)):
        line = [acc[idx],val_acc[idx],loss[idx],val_loss[idx], precision[idx], recall[idx]]        
        writer.writerow(line)
